In [15]:
import numpy as np
import random
import pickle
from simple_custom_taxi_env import SimpleTaxiEnv
import time

In [16]:
np.bool8 = np.bool_

env_config = {
    "fuel_limit": 5000
}
render = False
hyperparameters = {
    "alpha": 0.001,
	"gamma": 0.99,
	"epsilon_start": 1.0, 
	"epsilon_end": 0.1,
	"decay_rate": 0.999,
	"episodes": 1500,
 	"max_steps": 2000
}

In [ ]:
q_table = {}
epsilon = hyperparameters["epsilon_start"]

def get_state(obs, target_loc=None):
	stations = [[0, 0], [0, 4], [4, 0], [4,4]]
	taxi_row, taxi_col, stations[0][0],stations[0][1] ,stations[1][0],stations[1][1],stations[2][0],stations[2][1],stations[3][0],stations[3][1],obstacle_north, obstacle_south, obstacle_east, obstacle_west, passenger_look, destination_look = obs
	stations = [tuple(i) for i in stations]	

	assert target_loc is not None
	x_dir = target_loc[0] - taxi_row
	y_dir = target_loc[1] - taxi_col
	x_dir = 0 if x_dir == 0 else x_dir // abs(x_dir)
	y_dir = 0 if y_dir == 0 else y_dir // abs(y_dir)
	return (x_dir, y_dir, obstacle_north, obstacle_south, obstacle_east, obstacle_west, has_picked_up)
  
def get_action(obs):
	"""
	# Selects the best action using the trained Q-table.
	"""
	if np.random.uniform(0, 1) < epsilon:
		action = np.random.choice(action_nums)  # Random action
	else:
		action = np.argmax(q_table[get_state(obs)])  # Greedy action
	return action

def is_in_station(obs):
	"""
	# Checks if the taxi is in a station.
	"""
	stations = [[0, 0], [0, 4], [4, 0], [4,4]]
	taxi_row, taxi_col,stations[0][0],stations[0][1] ,stations[1][0],stations[1][1],stations[2][0],stations[2][1],stations[3][0],stations[3][1],obstacle_north, obstacle_south, obstacle_east, obstacle_west, passenger_look, destination_look = obs
	stations = [tuple(i) for i in stations]
	return (taxi_row, taxi_col) in stations

env = SimpleTaxiEnv(**env_config)
action_nums = 6
rewards_per_episode = []

obs, _ = env.reset()
total_reward = 0
done = False
step_count = 0
stations = [(0, 0), (0, 4), (4, 0), (4,4)]

taxi_row, taxi_col, _,_,_,_,_,_,_,_,obstacle_north, obstacle_south, obstacle_east, obstacle_west, passenger_look, destination_look = obs

if render:
	env.render_env((taxi_row, taxi_col),
					action=None, step=step_count, fuel=env.current_fuel)
	time.sleep(0.5)
 

for episode in range(hyperparameters["episodes"]):
	get_state.passenger_loc, get_state.destination_loc = None, None
	obs, _ = env.reset()
	done = False
	total_reward = 0
	step_count = 0
 
	destination = None
	visited = []
	has_picked_up = False
 
	taxi_row, taxi_col, _,_,_,_,_,_,_,_,obstacle_north, obstacle_south, obstacle_east, obstacle_west, passenger_look, destination_look = obs
	stations = [(obs[2], obs[3]), (obs[4], obs[5]), (obs[6], obs[7]), (obs[8], obs[9])]
	target_loc = stations[0]
	state = get_state(obs, target_loc)
	
	while not done:	
		if state not in q_table:
			q_table[state] = np.zeros(action_nums)
   
		taxi_row, taxi_col, _,_,_,_,_,_,_,_,obstacle_north, obstacle_south, obstacle_east, obstacle_west, passenger_look, destination_look = obs
		if np.random.uniform(0, 1) < epsilon:
			action_probs = np.ones(action_nums) / action_nums
			# if obstacle_south:
			# 	action_probs[0] = 0
			# if obstacle_north:
			# 	action_probs[1] = 0
			# if obstacle_east:
			# 	action_probs[2] = 0
			# if obstacle_west:
			# 	action_probs[3] = 0	
			# if not passenger_look or has_picked_up or not is_in_station(obs):
			# 	action_probs[4] = 0
			# if not destination_look or not has_picked_up or not is_in_station(obs):
			# 	action_probs[5] = 0
			# action_probs = action_probs / np.sum(action_probs)
			action = np.random.choice(action_nums, p=action_probs)  # Random action
		else:
			action = np.argmax(q_table[state])  # Greedy action
		
		if not has_picked_up and passenger_look and is_in_station(obs) and action == 4:
			has_picked_up = True
		if has_picked_up and destination_look and is_in_station(obs) and action == 5:
			done = True
   
		shaped_reward = 0
		x_dir = target_loc[0] - taxi_row
		y_dir = target_loc[1] - taxi_col
		x_dir = 0 if x_dir == 0 else x_dir // abs(x_dir)
		y_dir = 0 if y_dir == 0 else y_dir // abs(y_dir)
		if x_dir == 1 and action == 1 and not obstacle_north:
			shaped_reward = 1000
		if x_dir == -1 and action == 0 and not obstacle_south:
			shaped_reward = 1000
		if y_dir == 1 and action == 3 and not obstacle_west:
			shaped_reward = 1000
		if y_dir == -1 and action == 2 and not obstacle_east:
			shaped_reward = 1000
		if action == 4 and passenger_look and not has_picked_up:
			shaped_reward = 1000
		if action == 5 and destination_look and has_picked_up:
			shaped_reward = 1000
		if obstacle_south and action == 0:
			shaped_reward = -10000
		if obstacle_north and action == 1:
			shaped_reward = -10000
		if obstacle_east and action == 2:
			shaped_reward = -10000
		if obstacle_west and action == 3:
			shaped_reward = -10000
		if (not passenger_look or has_picked_up or not is_in_station(obs)) and action == 4:
			shaped_reward = -10000
		if (not destination_look or not has_picked_up or not is_in_station(obs)) and action == 5:
			shaped_reward = -10000
   
		next_obs, reward, done, _ = env.step(action)
		taxi_row, taxi_col, _,_,_,_,_,_,_,_,obstacle_north, obstacle_south, obstacle_east, obstacle_west, passenger_look, destination_look = next_obs
  
		if is_in_station(next_obs) and (taxi_row, taxi_col) not in visited:
			visited.append((taxi_row, taxi_col))
			if destination_look:
				destination = (taxi_row, taxi_col)
			if has_picked_up and destination is not None:
				target_loc = destination
			else:
				# choose a station that has not been visited yet
				# print (visited, (taxi_row, taxi_col))
				for station in stations:
					if station not in visited:
						target_loc = station
						break	
				# print (target_loc)

		# if done:
		# 	if not has_picked_up:
		# 		# shaped_reward = -100
		# 		# print ("not picked up", next_obs, action, has_picked_up, destination)
		# 		done = False
		# 	if has_picked_up and not destination_look:
		# 		# shaped_reward = -100
		# 		# print ("not destination")
		# 		done = False
		# 	if has_picked_up and destination_look:
		# 		# print ("done")
		# 		shaped_reward = 2000
		
		total_reward += reward
		reward += shaped_reward
		if next_obs not in q_table:
			next_state = get_state(next_obs, target_loc)
			q_table[next_state] = np.zeros(action_nums)
		q_table[state][action] += hyperparameters["alpha"] * (reward + hyperparameters["gamma"] * np.max(q_table[next_state]) - q_table[state][action])
		
		step_count += 1
		obs = next_obs
		state = next_state
   
	
		if render:
			env.render_env((taxi_row, taxi_col),
							action=action, step=step_count, fuel=env.current_fuel)
	# print (step_count)
	rewards_per_episode.append(total_reward)
	epsilon = max(hyperparameters["epsilon_end"], epsilon * hyperparameters["decay_rate"])
	if (episode + 1) % 20 == 0:
		avg_reward = np.mean(rewards_per_episode[-20:])
		print(f'Episode {episode + 1}/{hyperparameters["episodes"]}, Avg Reward: {avg_reward:.4f}, Epsilon: {epsilon:.3f}')
		# print ([np.argmax(i) for i in q_table.values()])
		print (len(q_table))
  
filename = "q_table.pkl"

# 儲存 Q-table
with open(filename, "wb") as f:
    pickle.dump(q_table, f)

print(f"Q-table 已儲存至 {filename}")

Episode 20/1500, Avg Reward: 20.0550, Epsilon: 0.980
141
Episode 40/1500, Avg Reward: 7.7800, Epsilon: 0.961
161
Episode 60/1500, Avg Reward: -2.2800, Epsilon: 0.942
163
Episode 80/1500, Avg Reward: 2.4350, Epsilon: 0.923
165
Episode 100/1500, Avg Reward: -29.5050, Epsilon: 0.905
166
Episode 120/1500, Avg Reward: -28.2350, Epsilon: 0.887
166
Episode 140/1500, Avg Reward: -84.3900, Epsilon: 0.869
166
Episode 160/1500, Avg Reward: -37.8350, Epsilon: 0.852
166
Episode 180/1500, Avg Reward: -170.1750, Epsilon: 0.835
166
Episode 200/1500, Avg Reward: -126.1950, Epsilon: 0.819
166
Episode 220/1500, Avg Reward: -221.6550, Epsilon: 0.802
167
Episode 240/1500, Avg Reward: -268.6150, Epsilon: 0.787
167
Episode 260/1500, Avg Reward: -275.3150, Epsilon: 0.771
167
Episode 280/1500, Avg Reward: -335.0800, Epsilon: 0.756
168
Episode 300/1500, Avg Reward: -327.0200, Epsilon: 0.741
168
Episode 320/1500, Avg Reward: -500.6350, Epsilon: 0.726
168
Episode 340/1500, Avg Reward: -710.1000, Epsilon: 0.712
16